In [1]:
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
day = datetime.datetime.strftime(datetime.datetime.now(), "%Y_%m_%d")
day

'2022_09_11'

In [3]:

! rm -rf "eqList/eqList"$day".xls"
! cp "/Users/ivan/Downloads/eqList"$day".xls" "eqList/eqList"$day".xls"
! ls -l "eqList/"

aeta = f"eqList/eqList{day}.xls"
aeta

total 73280
-rw-r--r--@ 1 ivan  staff   565464  7 30 18:56 eqList2022_07_30.xlsx
-rw-r--r--@ 1 ivan  staff   566439  8  9 01:05 eqList2022_08_09.xlsx
-rw-r--r--@ 1 ivan  staff  3567272  8 11 11:27 eqList2022_08_11.xls
-rw-r--r--@ 1 ivan  staff   566428  8 11 00:45 eqList2022_08_11.xlsx
-rw-r--r--@ 1 ivan  staff  3568288  8 12 16:56 eqList2022_08_12.xls
-rw-r--r--@ 1 ivan  staff  3569648  8 13 15:09 eqList2022_08_13.xls
-rw-r--r--@ 1 ivan  staff  3573668  8 15 16:06 eqList2022_08_15.xls
-rw-r--r--@ 1 ivan  staff  3575011  8 16 18:58 eqList2022_08_16.xls
-rw-r--r--@ 1 ivan  staff  3576341  8 19 17:36 eqList2022_08_19.xls
-rw-r--r--@ 1 ivan  staff  3581086  8 26 10:12 eqList2022_08_26.xls
-rw-r--r--@ 1 ivan  staff  3585174  9  2 01:05 eqList2022_09_02.xls
-rw-r--r--@ 1 ivan  staff  3590551  9  5 16:37 eqList2022_09_05.xls
-rw-r--r--@ 1 ivan  staff  3603008  9 11 16:27 eqList2022_09_11.xls


'eqList/eqList2022_09_11.xls'

In [4]:
N = 7
T = 10
# 经度
lonL, lonH = 98, 107
# 纬度
latL, latH = 22, 34 


In [5]:

import xml
DOMTree = xml.dom.minidom.parse(aeta).documentElement
Row = DOMTree.getElementsByTagName("Row")

cols, data = [], []
for nRow, iRow in enumerate(Row):
    if nRow == 0:
        for jRow in iRow.getElementsByTagName("Cell"):
            cols.append(jRow.childNodes[0].childNodes[0].data)
    else:
        idata = []
        for jRow in iRow.getElementsByTagName("Cell"):
            idata.append(jRow.childNodes[0].childNodes[0].data)
        data.append([float(dj) if nj in [1,2,3,4] else str(dj) for nj ,dj in enumerate(idata)])

data = pd.DataFrame(data, columns=cols)
print(data.dtypes)
data.head(T)


发震时刻       object
震级(M)     float64
纬度(°)     float64
经度(°)     float64
深度(千米)    float64
参考位置       object
dtype: object


,发震时刻,震级(M),纬度(°),经度(°),深度(千米),参考位置
0,2022-09-11 07:46:59,7.6,-6.30,146.55,70.0,巴布亚新几内亚
1,2022-09-11 07:10:43,6.2,-1.30,98.45,30.0,印尼苏门答腊岛南部海域
2,2022-09-11 04:50:51,3.1,36.02,77.91,10.0,新疆和田地区皮山县
3,2022-09-10 16:53:38,3.1,34.74,79.93,10.0,西藏阿里地区日土县
4,2022-09-10 13:51:06,3.7,46.08,90.20,13.0,新疆阿勒泰地区青河县
5,2022-09-10 10:01:06,3.7,37.74,101.46,9.0,青海海北州门源县
6,2022-09-10 08:05:15,6.0,-2.30,138.20,40.0,印尼巴布亚省
7,2022-09-10 07:31:50,6.2,-2.30,138.20,40.0,印尼巴布亚省
8,2022-09-09 11:08:43,3.6,33.60,81.72,10.0,西藏阿里地区日土县
9,2022-09-09 08:04:11,3.1,38.45,76.81,20.0,新疆喀什地区莎车县


In [6]:
def regeo(location):
    try:
        import json
        import requests

        host = 'https://regeo.market.alicloudapi.com'
        path = '/v3/geocode/regeo'
        appcode = '830fa522a512421eaa202bb80afe8921'
        querys = f'location={location}' # 经度在前，纬度在后
        with requests.get(
            f"{host}{path}?{querys}", 
            headers={'Authorization': f'APPCODE {appcode}'}
        ) as r:
            j = json.loads(r.content.decode())["regeocode"]["formatted_address"]
            return j
    except:
        return "NaN"

In [7]:

Kdata = []
for STATION in tqdm(data.head(T)["参考位置"]):
    # idata
    idata = data[data["参考位置"] == STATION][["参考位置", "发震时刻"]]
    idata.rename({
        "参考位置": "初次参考位置", 
        "发震时刻": "初次发震时刻",
    }, axis=1, inplace=True)
    idata["K"] = 1
    # print(idata.shape)

    # jdata
    jdata = data[
        (data["参考位置"] != STATION) & 
        (data["经度(°)"] >= lonL) & (data["经度(°)"] <= lonH) &
        (data["纬度(°)"] >= latL) & (data["纬度(°)"] <= latH) &
        # (data["震级(M)"] >= 3.5) &
        True
    ][["参考位置", "震级(M)", "纬度(°)", "经度(°)", "发震时刻"]]
    jdata["K"] = 1
    # print(jdata.shape)

    # k0data
    k0data = pd.merge(
        idata,
        jdata,
        on="K"
    )
    # print(k0data.shape)

    # k1data
    # 筛选近N天内
    k0data["gap-发震时刻"] = (
        pd.to_datetime(k0data["发震时刻"]) - 
        pd.to_datetime(k0data["初次发震时刻"])
    ).dt.days

    k1data = (k0data["gap-发震时刻"] >= 1) & (k0data["gap-发震时刻"] <= N)
    # print(pd.value_counts(k1data))

    k1data = k0data[k1data]
    # print(k1data.shape)

    # k2data
    k2data = k1data.groupby(
        ["初次参考位置", "初次发震时刻", "K"], 
        as_index=False
    ).agg({
        "震级(M)": "mean", # 特殊/余震
        "纬度(°)": "mean",
        "经度(°)": "mean",
    }).groupby(
        ["K"], 
        as_index=False
    ).agg(
        {
        "K": "sum",
        "震级(M)": "mean", 
        "纬度(°)": "mean",
        "经度(°)": "mean",
    })

    k2data["参考位置"] = STATION
    k2data["K%"] = k2data["K"]/idata.shape[0]
    k2data["震级C"] = k2data["震级(M)"].apply(lambda x: "" if x >= 3.5 else "❌")
    k2data["经度C"] = k2data["经度(°)"].apply(lambda x: "" if x >= lonL and x <= lonH else "❌")
    k2data["纬度C"] = k2data["纬度(°)"].apply(lambda x: "" if x >= latL and x <= latH else "❌")
    k2data["经纬度解析"] = [regeo(f"{_1},{_2}") for _1, _2 in zip(k2data["经度(°)"], k2data["纬度(°)"])]
    k2data["经度(°)"] = k2data["经度(°)"].apply(lambda x: f"{x:.6f}")
    k2data["纬度(°)"] = k2data["纬度(°)"].apply(lambda x: f"{x:.6f}")

    k2data = k2data[
        ['参考位置', '震级(M)', '震级C', '纬度(°)', '纬度C', '经度(°)', '经度C', 'K', 'K%','经纬度解析']
    ]
    # print(k2data)
    Kdata.append(k2data)

    # 
    del idata, jdata, k0data, k1data, k2data


100%|██████████| 10/10 [00:02<00:00,  3.45it/s]


In [8]:
endata = pd.merge(
    data.head(T).rename(
        {icol: f"#{icol}#" for icol in data.columns if icol != "参考位置"}, 
        axis=1
    ), 
    pd.concat(Kdata),
    on="参考位置"
).sort_values("#发震时刻#", ascending=False)

endata

,#发震时刻#,#震级(M)#,#纬度(°)#,#经度(°)#,#深度(千米)#,参考位置,震级(M),震级C,纬度(°),纬度C,经度(°),经度C,K,K%,经纬度解析
0,2022-09-11 07:46:59,7.6,-6.30,146.55,70.0,巴布亚新几内亚,3.186364,❌,28.217576,,103.023636,,11,0.234043,四川省凉山彝族自治州美姑县九口乡
1,2022-09-11 07:10:43,6.2,-1.30,98.45,30.0,印尼苏门答腊岛南部海域,3.830000,,27.942500,,103.629000,,10,0.285714,云南省昭通市永善县水竹乡魏家沟
2,2022-09-11 04:50:51,3.1,36.02,77.91,10.0,新疆和田地区皮山县,3.474295,❌,29.279077,,103.050814,,26,0.276596,四川省乐山市金口河区永和镇成昆线
3,2022-09-10 16:53:38,3.1,34.74,79.93,10.0,西藏阿里地区日土县,3.511111,,28.986667,,103.133333,,12,0.461538,四川省乐山市峨边彝族自治县金岩乡
4,2022-09-10 16:53:38,3.1,34.74,79.93,10.0,西藏阿里地区日土县,3.511111,,28.986667,,103.133333,,12,0.461538,四川省乐山市峨边彝族自治县金岩乡
7,2022-09-10 13:51:06,3.7,46.08,90.20,13.0,新疆阿勒泰地区青河县,3.100000,❌,25.170000,,99.490000,,1,0.250000,云南省大理白族自治州永平县杉阳镇
8,2022-09-10 10:01:06,3.7,37.74,101.46,9.0,青海海北州门源县,3.112500,❌,28.070417,,103.735000,,12,0.307692,云南省昭通市永善县马楠苗族彝族乡红鸡坪
9,2022-09-10 08:05:15,6.0,-2.30,138.20,40.0,印尼巴布亚省,3.188889,❌,28.482778,,104.796667,,3,0.230769,四川省宜宾市长宁县花滩镇环山埂
10,2022-09-10 08:05:15,6.0,-2.30,138.20,40.0,印尼巴布亚省,3.188889,❌,28.482778,,104.796667,,3,0.230769,四川省宜宾市长宁县花滩镇环山埂
11,2022-09-10 07:31:50,6.2,-2.30,138.20,40.0,印尼巴布亚省,3.188889,❌,28.482778,,104.796667,,3,0.230769,四川省宜宾市长宁县花滩镇环山埂


In [9]:
# 
day1 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=1), "%Y-%m-%d")
dayn = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=N), "%Y-%m-%d")
day7 = datetime.datetime.strftime(datetime.datetime.now() + datetime.timedelta(days=7), "%Y-%m-%d")
print(f">>> {day1} To {day7} / {dayn}")

message = f"""
# 有震预测 earthquake prediction
check_my_prediction(myToken, '{day1}', '{dayn}', 1, latitude=$_1, longitude=$_2, magnitude=$_3)
"""
endata = endata[endata["震级C"] != "❌"]
for _1, _2, _3 in zip(endata["纬度(°)"], endata["经度(°)"], endata["震级(M)"]):
    print(message
          .replace("$_1",f"{float(_1):.6f}")
          .replace("$_2",f"{float(_2):.6f}")
          .replace("$_3",f"{float(_3):.1f}")
         )

message = f"""
# 无震预测 No earthquake prediction
check_my_prediction(myToken, '{day1}', '{day7}', 0)
"""
print(message)
print()


>>> 2022-09-12 To 2022-09-18 / 2022-09-12

# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-12', 1, latitude=27.942500, longitude=103.629000, magnitude=3.8)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-12', 1, latitude=28.986667, longitude=103.133333, magnitude=3.5)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-12', 1, latitude=28.986667, longitude=103.133333, magnitude=3.5)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-12', 1, latitude=28.986667, longitude=103.133333, magnitude=3.5)


# 有震预测 earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-12', 1, latitude=28.986667, longitude=103.133333, magnitude=3.5)


# 无震预测 No earthquake prediction
check_my_prediction(myToken, '2022-09-12', '2022-09-18', 0)




In [10]:
! cp Test002.ipynb "back/Test002-"$day".ipynb"
! ls -l back/

total 392
-rw-r--r--@ 1 ivan  staff  34490  8 16 18:58 Test002-2022_08_16.ipynb
-rw-r--r--@ 1 ivan  staff  33275  8 19 17:36 Test002-2022_08_19.ipynb
-rw-r--r--@ 1 ivan  staff  33096  8 26 10:12 Test002-2022_08_26.ipynb
-rw-r--r--@ 1 ivan  staff  29782  9  2 01:05 Test002-2022_09_02.ipynb
-rw-r--r--@ 1 ivan  staff   8874  9  5 16:37 Test002-2022_09_05.ipynb
-rw-r--r--@ 1 ivan  staff  43740  9 11 16:27 Test002-2022_09_11.ipynb
